<a href="https://colab.research.google.com/github/palomovisk/CienciadeDados/blob/main/CalculoPIscattergather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,526 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,939 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd

In [ ]:
!apt install openmpi-bin openmpi-common libopenmpi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-common is already the newest version (4.1.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%%writefile CalculoPIscattergather-mpi.c

//Paloma de Sousa Bezerra
//10376795


#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

//Funcao para calcular pi
double calcula_pi(double *data, int local_n) {
    double sum = 0.0;
    for (int i = 0; i < local_n; i++) {
        double factor = ((int)data[i] % 2 == 0) ? 1.0 : -1.0;
        sum += factor / (2 * data[i] + 1);
    }
    return sum;
}

int main(int argc, char *argv[]) {
    int rank, size, n = 1000000;
    double *data = NULL, *local_data;
    double local_sum, *res_data;
    double pi = 0.0;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int local_n = n / size;
    local_data = (double *)malloc(local_n * sizeof(double));
    res_data = (double *)malloc(size * sizeof(double));

    // Processo raiz gera os dados
    if (rank == 0) {
        data = (double *)malloc(n * sizeof(double));
        for (int i = 0; i < n; i++) {
            data[i] = i;
        }
    }

    // Distribui os dados entre os processos
    MPI_Scatter(data, local_n, MPI_DOUBLE, local_data, local_n, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    // Cada processo calcula sua parte da série
    local_sum = calcula_pi(local_data, local_n);

    // Coleta os resultados parciais no processo raiz
    MPI_Gather(&local_sum, 1, MPI_DOUBLE, res_data, 1, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    // Processo raiz calcula o valor final de pi
    if (rank == 0) {
        double global_sum = 0.0;
        for (int i = 0; i < size; i++) {
            global_sum += res_data[i];
        }
        pi = 4.0 * global_sum;
        printf("Resultado de Pi: %.15f\n", pi);

        // Libera memória alocada
        free(data);
    }

    // Libera memória alocada
    free(local_data);
    free(res_data);

    MPI_Finalize();
    return 0;
}

Overwriting CalculoPIscattergather-mpi.c


In [ ]:
!mpicc -o CalculoPIscattergather-mpi CalculoPIscattergather-mpi.c

In [ ]:
!mpiexec  --allow-run-as-root --oversubscribe  -n 4 ./CalculoPIscattergather-mpi

Resultado de Pi: 3.141591653589781
